In [41]:
#similairty distance between two persons with respect to preference

def sim_distance(prefs, person1, person2):
    sum_of_squares = 0
    common_count = 0
    for movie in prefs[person1]:
        if movie in prefs[person2]:
            sum_of_squares = sum_of_squares + pow(prefs[person1][movie] - prefs[person2][movie], 2)
            common_count = common_count + 1

    if common_count == 0:
        return 0
    else:
        return 1/(1+sum_of_squares)
    

#Pearson similarity scoring between two users     
from math import sqrt
def sim_pearson(prefs, p1, p2):
    sum1 = sum(prefs[p1].values())
    sum2 = sum(prefs[p2].values())
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
    # Find the number of elements
    n=len(si)
    # if they are no ratings in common, return 0
    if n==0: return 0
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r


# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.

def topMatches(prefs, person, n=5, similarity=sim_distance):
     scores = [(similarity(prefs, person, other), other) for other in prefs]

     scores.sort(reverse=True)
     return scores[:n]
    
    
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: continue
        sim=similarity(prefs,person,other)
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

def loadMovieLens():
    # Get movie titles
    movies={}
    for line in open('u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    for line in open('u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs


prefs = loadMovieLens()


ApnaDeclaration= '''This is a code for movie recommendation. We have taken the data from 
https://grouplens.org/datasets/movielens/100k/. If you give a user id as an input, this will recommend 5 movies
for the user, that he has not seen.'''

print(ApnaDeclaration)

switch = int(input("Enter User Id: "))

print ("The recommendations for User "+ str(switch)+ " are:")
getRecommendations(prefs,str(switch))[0:10]

This is a code for movie recommendation. We have taken the data from 
https://grouplens.org/datasets/movielens/100k/. If you give a user id as an input, this will recommend 5 movies
for the user, that he has not seen.
Enter User Id: 15
The recommendations for User 15 are:


[(5.0, 'Two or Three Things I Know About Her (1966)'),
 (5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Leading Man, The (1996)'),
 (5.0, 'Great Day in Harlem, A (1994)')]